In [1]:
from sklearn.preprocessing import MinMaxScaler
import math
import scipy.stats as st
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [8]:
df = pd.read_csv("amazon_review.csv")

In [9]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote
0,A3SBTW3WS4IQSN,B007WTAJTO,NaN,"[0, 0]",No issues.,4.00000,Four Stars,1406073600,2014-07-23,138,0,0
1,A18K1ODH1I2MVB,B007WTAJTO,0mie,"[0, 0]","Purchased this for my device, it worked as adv...",5.00000,MOAR SPACE!!!,1382659200,2013-10-25,409,0,0
2,A2FII3I2MBMUIA,B007WTAJTO,1K3,"[0, 0]",it works as expected. I should have sprung for...,4.00000,nothing to really say....,1356220800,2012-12-23,715,0,0
3,A3H99DFEG68SR,B007WTAJTO,1m2,"[0, 0]",This think has worked out great.Had a diff. br...,5.00000,Great buy at this price!!! *** UPDATE,1384992000,2013-11-21,382,0,0
4,A375ZM4U047O79,B007WTAJTO,2&amp;1/2Men,"[0, 0]","Bought it with Retail Packaging, arrived legit...",5.00000,best deal around,1373673600,2013-07-13,513,0,0


In [10]:
df.shape

(4915, 12)

In [11]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
overall,4915.00000,4.58759,0.99685,1.00000,5.00000,5.00000,5.00000,5.00000
unixReviewTime,4915.00000,1379465001.66836,15818574.32275,1339200000.00000,1365897600.00000,1381276800.00000,1392163200.00000,1406073600.00000
day_diff,4915.00000,437.36704,209.43987,1.00000,281.00000,431.00000,601.00000,1064.00000
helpful_yes,4915.00000,1.31109,41.61916,0.00000,0.00000,0.00000,0.00000,1952.00000
total_vote,4915.00000,1.52146,44.12309,0.00000,0.00000,0.00000,0.00000,2020.00000


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4915 entries, 0 to 4914
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   reviewerID      4915 non-null   object 
 1   asin            4915 non-null   object 
 2   reviewerName    4914 non-null   object 
 3   helpful         4915 non-null   object 
 4   reviewText      4914 non-null   object 
 5   overall         4915 non-null   float64
 6   summary         4915 non-null   object 
 7   unixReviewTime  4915 non-null   int64  
 8   reviewTime      4915 non-null   object 
 9   day_diff        4915 non-null   int64  
 10  helpful_yes     4915 non-null   int64  
 11  total_vote      4915 non-null   int64  
dtypes: float64(1), int64(4), object(7)
memory usage: 460.9+ KB


In [13]:
df.isnull().sum()

reviewerID        0
asin              0
reviewerName      1
helpful           0
reviewText        1
overall           0
summary           0
unixReviewTime    0
reviewTime        0
day_diff          0
helpful_yes       0
total_vote        0
dtype: int64

Average Score (Ürünün ortalama puanı)

In [14]:
df["overall"].mean()

4.587589013224822

### Time Based Weighted Average (Tarihe Göre Ortalama Ürün Puanı Hesabı)

In [16]:
df.loc[df["day_diff"] <= df["day_diff"].quantile(0.25), "overall"].mean()

4.6957928802588995

In [17]:
df.loc[(df["day_diff"] > df["day_diff"].quantile(0.25))&(df["day_diff"] <= df["day_diff"].quantile(0.5)), "overall"].mean()

4.636140637775961

In [20]:
df.loc[(df["day_diff"] > df["day_diff"].quantile(0.5))&(df["day_diff"] <= df["day_diff"].quantile(0.75)), "overall"].mean()

4.571661237785016

In [19]:
df.loc[(df["day_diff"] > df["day_diff"].quantile(0.75)), "overall"].mean()

4.4462540716612375

Eski tarihlere doğru gidildikçe ürünün ortalama puanı düşüyor. Güncel tarihlerde daha yüksek olduğu gözlemleniyor. Bunun sebebi değerlendirmelere ve yorumlara göre satıcının kötü olan şeyleri düzeltme çabası olabilir.

In [22]:
def time_based_weighted_average(df, w1=28, w2=26, w3=24, w4=22):
    return (df.loc[df["day_diff"] <= df["day_diff"].quantile(0.25), "overall"].mean() *w1/100 + \
            df.loc[(df["day_diff"] > df["day_diff"].quantile(0.25))&(df["day_diff"] <= df["day_diff"].quantile(0.5)), "overall"].mean() *w2/100 + \
            df.loc[(df["day_diff"] > df["day_diff"].quantile(0.5))&(df["day_diff"] <= df["day_diff"].quantile(0.75)), "overall"].mean() *w3/100 + \
            df.loc[(df["day_diff"] > df["day_diff"].quantile(0.75)), "overall"].mean() * w4/100)

In [23]:
time_based_weighted_average(df)

4.595593165128118

In [24]:
df["overall"].mean()

4.587589013224822

#### Ürün için Ürün Detay Sayfasında Görüntülenecek 20 Review'i Belirleyelim.

In [25]:
# Yararlı olmayan yorumların sayısını buluyoruz.
df["helpful_no"] = df["total_vote"] - df["helpful_yes"]

In [26]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote,helpful_no
0,A3SBTW3WS4IQSN,B007WTAJTO,NaN,"[0, 0]",No issues.,4.00000,Four Stars,1406073600,2014-07-23,138,0,0,0
1,A18K1ODH1I2MVB,B007WTAJTO,0mie,"[0, 0]","Purchased this for my device, it worked as adv...",5.00000,MOAR SPACE!!!,1382659200,2013-10-25,409,0,0,0
2,A2FII3I2MBMUIA,B007WTAJTO,1K3,"[0, 0]",it works as expected. I should have sprung for...,4.00000,nothing to really say....,1356220800,2012-12-23,715,0,0,0
3,A3H99DFEG68SR,B007WTAJTO,1m2,"[0, 0]",This think has worked out great.Had a diff. br...,5.00000,Great buy at this price!!! *** UPDATE,1384992000,2013-11-21,382,0,0,0
4,A375ZM4U047O79,B007WTAJTO,2&amp;1/2Men,"[0, 0]","Bought it with Retail Packaging, arrived legit...",5.00000,best deal around,1373673600,2013-07-13,513,0,0,0


In [27]:
def score_up_down_diff(up, down):
    return up - down

In [28]:
def score_average_rating(up, down):
    if up + down == 0:
        return 0
    return up / (up + down)

In [29]:
def wilson_lower_bound(up, down, confidence=0.95):
    """
    Wilson Lower Bound Score hesapla

    - Bernoulli parametresi p için hesaplanacak güven aralığının alt sınırı WLB skoru olarak kabul edilir.
    - Hesaplanacak skor ürün sıralaması için kullanılır.
    - Not:
    Eğer skorlar 1-5 arasıdaysa 1-3 negatif, 4-5 pozitif olarak işaretlenir ve bernoulli'ye uygun hale getirilebilir.
    Bu beraberinde bazı problemleri de getirir. Bu sebeple bayesian average rating yapmak gerekir.

    Parameters
    ----------
    up: int
        up count
    down: int
        down count
    confidence: float
        confidence

    Returns
    -------
    wilson score: float

    """
    n = up + down
    if n == 0:
        return 0
    z = st.norm.ppf(1 - (1 - confidence) / 2)
    phat = 1.0 * up / n
    return (phat + z * z / (2 * n) - z * math.sqrt((phat * (1 - phat) + z * z / (4 * n)) / n)) / (1 + z * z / n)

In [37]:
df["score_pos_neg_diff"] = df.apply(lambda x: score_up_down_diff(x["helpful_yes"], x["helpful_no"]), axis=1)
df.sort_values("score_pos_neg_diff", ascending=False).head(20)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote,helpful_no,score_pos_neg_diff
2031,A12B7ZMXFI6IXY,B007WTAJTO,"Hyoun Kim ""Faluzure""","[1952, 2020]",[[ UPDATE - 6/19/2014 ]]So my lovely wife boug...,5.00000,UPDATED - Great w/ Galaxy S4 & Galaxy Tab 4 10...,1367366400,2013-01-05,702,1952,2020,68,1884
4212,AVBMZZAFEKO58,B007WTAJTO,SkincareCEO,"[1568, 1694]",NOTE: please read the last update (scroll to ...,1.00000,1 Star reviews - Micro SDXC card unmounts itse...,1375660800,2013-05-08,579,1568,1694,126,1442
3449,AOEAD7DPLZE53,B007WTAJTO,NLee the Engineer,"[1428, 1505]",I have tested dozens of SDHC and micro-SDHC ca...,5.00000,Top of the class among all (budget-priced) mic...,1348617600,2012-09-26,803,1428,1505,77,1351
317,A1ZQAQFYSXL5MQ,B007WTAJTO,"Amazon Customer ""Kelly""","[422, 495]","If your card gets hot enough to be painful, it...",1.00000,"Warning, read this!",1346544000,2012-02-09,1033,422,495,73,349
3981,A1K91XXQ6ZEBQR,B007WTAJTO,"R. Sutton, Jr. ""RWSynergy""","[112, 139]",The last few days I have been diligently shopp...,5.00000,"Resolving confusion between ""Mobile Ultra"" and...",1350864000,2012-10-22,777,112,139,27,85
4596,A1WTQUOQ4WG9AI,B007WTAJTO,"Tom Henriksen ""Doggy Diner""","[82, 109]",Hi:I ordered two card and they arrived the nex...,1.00000,Designed incompatibility/Don't support SanDisk,1348272000,2012-09-22,807,82,109,27,55
1835,A1J6VSUM80UAF8,B007WTAJTO,goconfigure,"[60, 68]",Bought from BestBuy online the day it was anno...,5.00000,I own it,1393545600,2014-02-28,283,60,68,8,52
4672,A2DKQQIZ793AV5,B007WTAJTO,Twister,"[45, 49]",Sandisk announcement of the first 128GB micro ...,5.00000,Super high capacity!!! Excellent price (on Am...,1394150400,2014-07-03,158,45,49,4,41
4306,AOHXKM5URSKAB,B007WTAJTO,Stellar Eller,"[51, 65]","While I got this card as a ""deal of the day"" o...",5.00000,Awesome Card!,1339200000,2012-09-06,823,51,65,14,37
315,A2J26NNQX6WKAU,B007WTAJTO,"Amazon Customer ""johncrea""","[38, 48]",Bought this card to use with my Samsung Galaxy...,5.00000,Samsung Galaxy Tab2 works with this card if re...,1344816000,2012-08-13,847,38,48,10,28


In [38]:
df["score_average_rating"] = df.apply(lambda x: score_average_rating(x["helpful_yes"], x["helpful_no"]), axis=1)

In [39]:
df.sort_values("score_average_rating", ascending=False).head(20)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote,helpful_no,score_pos_neg_diff,score_average_rating
4277,A35KXSU6AD1481,B007WTAJTO,S. Q.,"[1, 1]",I have a galaxy note II and after rooting I no...,5.00000,Perfect!!,1355875200,2012-12-19,719,1,1,0,1,1.00000
2881,A3VSG5X7GPNNW6,B007WTAJTO,Lou Thomas,"[1, 1]",The Nexus One is listed as supporting a maximu...,5.00000,Nexus One Loves This Card!,1349049600,2012-01-10,1063,1,1,0,1,1.00000
1073,A2ZXEKQ2OBZLEE,B007WTAJTO,C. Sanchez,"[1, 1]",I used it with my Samsung S4 and it works grea...,5.00000,Tons of space for phone,1376352000,2013-08-13,482,1,1,0,1,1.00000
445,AIWBDRNBODLEA,B007WTAJTO,"Apache ""Elizabeth""","[1, 1]",This is exactly what I was looking for to upgr...,4.00000,Amazon Great Prices,1387324800,2013-12-18,355,1,1,0,1,1.00000
3923,A2PH4RGYVR34L,B007WTAJTO,Rock Your Roots,"[1, 1]","It's a SanDisk, so what more is there to say? ...",5.00000,What more to say?,1388361600,2013-12-30,343,1,1,0,1,1.00000
435,AUH8I22ITG020,B007WTAJTO,Anthony L cate,"[1, 1]",This is working great in my AT&T Galaxy Note. ...,5.00000,Love the extra storage,1343088000,2012-07-24,867,1,1,0,1,1.00000
2901,A28TRYU3FJ039C,B007WTAJTO,luis,"[1, 1]",Not a good typer or speller :) here is what I ...,5.00000,Awesome and fast card :),1368403200,2013-05-13,574,1,1,0,1,1.00000
2204,AANX2UN8NPE22,B007WTAJTO,"jbwam ""jbwam""","[1, 1]",I just called Sandisk and they say they have a...,2.00000,Sandisk will replace failures due to bad batch...,1371168000,2013-06-14,542,1,1,0,1,1.00000
2206,A3KO3964CNP0XN,B007WTAJTO,JCBiker,"[1, 1]",I bought this for my garmin virb action cam. ...,5.00000,Great card,1383177600,2013-10-31,403,1,1,0,1,1.00000
3408,A20WUUD9EDWY4N,B007WTAJTO,"Neng Vang ""Neng2012""","[1, 1]",Very good card and still working now in my car...,5.00000,working no problem,1374710400,2013-07-25,501,1,1,0,1,1.00000


In [40]:
df["wilson_lower_bound"] = df.apply(lambda x: wilson_lower_bound(x["helpful_yes"], x["helpful_no"]), axis=1)
df.sort_values("wilson_lower_bound", ascending=False)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote,helpful_no,score_pos_neg_diff,score_average_rating,wilson_lower_bound
2031,A12B7ZMXFI6IXY,B007WTAJTO,"Hyoun Kim ""Faluzure""","[1952, 2020]",[[ UPDATE - 6/19/2014 ]]So my lovely wife boug...,5.00000,UPDATED - Great w/ Galaxy S4 & Galaxy Tab 4 10...,1367366400,2013-01-05,702,1952,2020,68,1884,0.96634,0.95754
3449,AOEAD7DPLZE53,B007WTAJTO,NLee the Engineer,"[1428, 1505]",I have tested dozens of SDHC and micro-SDHC ca...,5.00000,Top of the class among all (budget-priced) mic...,1348617600,2012-09-26,803,1428,1505,77,1351,0.94884,0.93652
4212,AVBMZZAFEKO58,B007WTAJTO,SkincareCEO,"[1568, 1694]",NOTE: please read the last update (scroll to ...,1.00000,1 Star reviews - Micro SDXC card unmounts itse...,1375660800,2013-05-08,579,1568,1694,126,1442,0.92562,0.91214
317,A1ZQAQFYSXL5MQ,B007WTAJTO,"Amazon Customer ""Kelly""","[422, 495]","If your card gets hot enough to be painful, it...",1.00000,"Warning, read this!",1346544000,2012-02-09,1033,422,495,73,349,0.85253,0.81858
4672,A2DKQQIZ793AV5,B007WTAJTO,Twister,"[45, 49]",Sandisk announcement of the first 128GB micro ...,5.00000,Super high capacity!!! Excellent price (on Am...,1394150400,2014-07-03,158,45,49,4,41,0.91837,0.80811
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1712,A1N5UN0DUW3P42,B007WTAJTO,G. Abbott,"[0, 0]",I use this in my Samsung Galaxy S5 to store my...,5.00000,Works great,1403568000,2014-06-24,167,0,0,0,0,0.00000,0.00000
1711,A2OBSXOTTH3XPD,B007WTAJTO,G68,"[0, 0]",awesome! Worked well with the Surface 2 tablet...,5.00000,No issues - Good micro,1396915200,2014-08-04,126,0,0,0,0,0.00000,0.00000
1710,A3T0DDOQYML4N5,B007WTAJTO,G0Horns!,"[0, 0]",Its amazing the amount of storage we are putti...,4.00000,So far so good,1369612800,2013-05-27,560,0,0,0,0,0.00000,0.00000
1709,A2WB2KKS7MXTCW,B007WTAJTO,g0estoeleven,"[0, 0]",This card does exactly what it should. Class-...,3.00000,Good value for its class but no speed demon,1375488000,2013-03-08,640,0,0,0,0,0.00000,0.00000
